## <span style="color: Gold"> **chroma DB**

- 벡터 DB : 벡터 + DB
- DB : 쿼리가 정확히 일치해야 결과를 출력  ex. 대한민국 수도 <---> 한국의 수도 : 다르게 인식
- ChromaDB : 단어의 의미와 맥락을 기반으로 데이터를 찾음  ex. 대한민국 수도 <---> 한국의 수도 : 유사함을 찾음

In [ ]:
# 임베딩을 따로 정의안하고 기본으로 사용한 chromadb  

import chromadb
# 1. 클라이언트 생성 (메모리에만 저장)
client = chromadb.Client() #--> 인메모리 방식 / 객체를 만들었다라고 생각

# 2. 컬렉션 생성  (데이터베이스에 스키마, 테이블.. 같은걸 만들었다 생각)
collection = client.get_or_create_collection('my_collection')  # get_or_create_collection 있으면 가져오고 없으면 만들어라

# 3. 데이터를 추가
import uuid
collection.add(
    documents = ['화면을 보여주는 모니터', '글자를 입력하는 키보드', '커서를 움직이는 마우스'], # 벡터가 될 문장
    metadatas = [{'category' : 'display'}, {'category':'input'}, {'category': 'input'}], # 문장들과 같이 사용할 .. ? 인덱스화 됐다  #필수아니고, 옵션.
    ids = [str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4())]       # 필수, 각각의 고유번호. ids는 str만 허용해서 str()씌워줌
)

# 4. 질문하기 (의미기반)
results = collection.query(
    query_texts=['숫자를 입력하는 기계'],
    n_results= 1 #결과 하나만
)

results

# 결과가 좋지 않음. chromadb가 한국어가 별로임 # default languge 를 사용해서.
    # query_texts=['뭔가를 보여주는 기계'], ---> 출력  'documents': [['커서를 움직이는 마우스']],
    # query_texts=['숫자를 입력하는 기계'] ---> 출력 'documents': [['글자를 입력하는 키보드']],




{'ids': [['e901dc77-59dd-44b3-b9ba-fd24767928b7']],
 'embeddings': None,
 'documents': [['글자를 입력하는 키보드']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'category': 'input'}]],
 'distances': [[0.626086950302124]]}

### <span style="color: Gold"> **chromaDB 한국어 전용**


- chromaDB는 영어 전용 모델이 탑재되어 한글에 불리함.
- 다국어나 한국어 전용 모델을 사용 

In [ ]:
### openai 를 사용한 임베딩!!! 

import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os
from dotenv import load_dotenv
load_dotenv()

# 1. 클라이언트 생성 (메모리에만 저장)

client = chromadb.Client() #--> 인메모리 방식 / 객체를 만들었다라고 생각
embed_fn = OpenAIEmbeddingFunction( 
            api_key= os.environ['OPENAI_API_KEY'],
            model_name="text-embedding-3-small"
        )
    

# 2. 컬렉션 생성  (데이터베이스에 스키마, 테이블.. 같은걸 만들었다 생각)
collection = client.get_or_create_collection('my_collection2', embedding_function=embed_fn)  




# 3. 데이터를 추가
import uuid
collection.add(
    documents = ['화면을 보여주는 모니터', '글자를 입력하는 키보드', '커서를 움직이는 마우스'], # 벡터가 될 문장
    metadatas = [{'category' : 'display'}, {'category':'input'}, {'category': 'input'}], # 문장들과 같이 사용할 .. ? 인덱스화 됐다  #필수아니고, 옵션.
    ids = [str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4())]       # 필수, 각각의 고유번호. ids는 str만 허용해서 str()씌워줌
)

# 4. 질문하기 (의미기반)
results = collection.query(
    query_texts=['뭔가를 보여주는 기계'],
    n_results= 1 #결과 하나만
)

results

# 결과 나아짐
    # query_texts=['뭔가를 보여주는 기계'], ---> 출력   'documents': [['화면을 보여주는 모니터']],




{'ids': [['57c342dd-1b96-4ced-a6cb-0d9276c1060a']],
 'embeddings': None,
 'documents': [['화면을 보여주는 모니터']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'category': 'display'}]],
 'distances': [[0.5194305777549744]]}

In [ ]:
### Hugging Face 의  model_name= "paraphrase-multilingual-MiniLM-L12-v2" 임베딩 사용



import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os
from dotenv import load_dotenv
load_dotenv()

# 1. 클라이언트 생성 (메모리에만 저장)

client = chromadb.Client() 

from chromadb.utils import embedding_functions
embed_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name= "paraphrase-multilingual-MiniLM-L12-v2"
)
    

# 2. 컬렉션 생성  (데이터베이스에 스키마, 테이블.. 같은걸 만들었다 생각)
collection = client.get_or_create_collection('my_collection3', embedding_function=embed_fn)  




# 3. 데이터를 추가
import uuid
collection.add(
    documents = ['화면을 보여주는 모니터', '글자를 입력하는 키보드', '커서를 움직이는 마우스'], # 벡터가 될 문장
    metadatas = [{'category' : 'display'}, {'category':'input'}, {'category': 'input'}], # 문장들과 같이 사용할 .. ? 인덱스화 됐다  #필수아니고, 옵션.
    ids = [str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4())]       # 필수, 각각의 고유번호. ids는 str만 허용해서 str()씌워줌
)

# 4. 질문하기 (의미기반)
results = collection.query(
    query_texts=['뭔가를 보여주는 기계'],
    n_results= 1 #결과 하나만
)

results

# 결과 좋지 않음
    # query_texts=['뭔가를 보여주는 기계'], ---> 출력  'documents': [['커서를 움직이는 마우스']],



{'ids': [['d925c97c-7a28-42b2-ab09-822140348670']],
 'embeddings': None,
 'documents': [['커서를 움직이는 마우스']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'category': 'input'}]],
 'distances': [[0.2587941288948059]]}